In [1]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 15, 2022...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


\--- Day 15: Beacon Exclusion Zone ---
----------

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable *sensors* that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source *beacon*. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can *determine the position of a beacon precisely*; however, sensors can only lock on to the one beacon *closest to the sensor* as measured by the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry). (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:

```
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3

```

So, consider the sensor at `2,18`; the closest beacon to it is at `-2,15`. For the sensor at `9,16`, the closest beacon to it is at `10,16`.

Drawing sensors as `S` and beacons as `B`, the above arrangement of sensors and beacons looks like this:

```
               1    1    2    2
     0    5    0    5    0    5
 0 ....S.......................
 1 ......................S.....
 2 ...............S............
 3 ................SB..........
 4 ............................
 5 ............................
 6 ............................
 7 ..........S.......S.........
 8 ............................
 9 ............................
10 ....B.......................
11 ..S.........................
12 ............................
13 ............................
14 ..............S.......S.....
15 B...........................
16 ...........SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....

```

This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at `8,7`:

```
               1    1    2    2
     0    5    0    5    0    5
-2 ..........#.................
-1 .........###................
 0 ....S...#####...............
 1 .......#######........S.....
 2 ......#########S............
 3 .....###########SB..........
 4 ....#############...........
 5 ...###############..........
 6 ..#################.........
 7 .#########S#######S#........
 8 ..#################.........
 9 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....

```

This sensor's closest beacon is at `2,10`, and so you know there are no beacons that close or closer (in any positions marked `#`).

None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it *isn't*. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where `y=10`, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

```
                 1    1    2    2
       0    5    0    5    0    5
 9 ...#########################...
10 ..####B######################..
11 .###S#############.###########.

```

In this example, in the row where `y=10`, there are `*26*` positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. *In the row where `y=2000000`, how many positions cannot contain a beacon?*


In [2]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    lines = [x.strip() for x in f.readlines()]

# Verify parse
pprint(lines[:6])

['Sensor at x=325337, y=2568863: closest beacon is at x=-518661, y=2000000',
 'Sensor at x=3988825, y=837820: closest beacon is at x=4305648, y=2127118',
 'Sensor at x=1611311, y=2053174: closest beacon is at x=2827226, y=1579510',
 'Sensor at x=101890, y=3940049: closest beacon is at x=955472, y=3457514',
 'Sensor at x=3962702, y=2558425: closest beacon is at x=4226981, y=2604726',
 'Sensor at x=2957890, y=2160813: closest beacon is at x=2827226, y=1579510']


In [3]:
import re
pattern = r"x=([-\d]+), y=([-\d]+):.*x=([-\d]+), y=([-\d]+)"

reports = [re.findall(pattern, line)[0] for line in lines]
reports = [[int(_) for _ in report] for report in reports]
pprint(reports[:6])

[[325337, 2568863, -518661, 2000000],
 [3988825, 837820, 4305648, 2127118],
 [1611311, 2053174, 2827226, 1579510],
 [101890, 3940049, 955472, 3457514],
 [3962702, 2558425, 4226981, 2604726],
 [2957890, 2160813, 2827226, 1579510]]


In [4]:
# For each position, iterate over sensors. Is this position within {sensor:beacon} distance of sensor?

def manhatten_distance(x1,y1,x2,y2):
    return abs(x2 - x1) + abs(y2 - y1)

assert manhatten_distance(8,7,2,10) == 9

def is_within_range(x, y, reports):
    for report in reports:
        sb_dist = manhatten_distance(*report)
        s_dist = manhatten_distance(x,y,report[0], report[1])
        b_dist = manhatten_distance(x,y,report[2], report[3])
        if s_dist == 0 or b_dist == 0: return False
        if s_dist <= sb_dist: return True
    return False

In [5]:
min_x = min([min(r[0],r[2]) for r in reports])
max_x = max([max(r[0],r[2]) for r in reports])
print(f"x  = {min_x} :: {max_x}")

count = 0
y=2_000_000
for x in range(min_x-100_000, max_x + 100_000):
    if is_within_range(x,y,reports):
        #print(x)
        count += 1

count

x  = -518661 :: 4305648


4924308

In [6]:

answer1 = 123
print("answer 1:", answer1)

answer 1: 123


----

In [7]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 15, 2022...
Saving puzzle description to "README.md"...
Done!


\--- Part Two ---
----------


Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have `x` and `y` coordinates each no lower than `0` and no larger than `4000000`.

To isolate the distress beacon's signal, you need to determine its *tuning frequency*, which can be found by multiplying its `x` coordinate by `4000000` and then adding its `y` coordinate.

In the example above, the search space is smaller: instead, the `x` and `y` coordinates can each be at most `20`. With this reduced search area, there is only a single position that could have a beacon: `x=14, y=11`. The tuning frequency for this distress beacon is `*56000011*`.

Find the only possible position for the distress beacon. *What is its tuning frequency?*


In [8]:
sensors = [[r[0],r[1]] for r in reports]
beacons = [[r[2],r[3]] for r in reports]

for i, report in enumerate(reports):
    dist = manhatten_distance(*report)+1
    print(f"{i}/{len(reports)}: searching at dist={dist} from sensor {report[:2]}")
    
    # Generate perimeter coordinates
    x1=[_ for _ in range(-dist,dist)]
    x2=[_ for _ in range(dist,-dist,-1)]
    y1=[dist-abs(x) for x in x1]
    y2=[-abs(dist-abs(x)) for x in x2]

    perim = zip(x1+x2, y1+y2)

    for x,y in perim:
        x = report[0] + x
        y = report[1] + y

        # Check only within search space
        if x < 0 or y < 0 or x > 4_000_000 or y > 4_000_000:
            continue

        # Check that point doesn't have a beacon already
        if [x,y] in beacons:
            continue
        
        # Check point against other boundaries
        if not is_within_range(x, y, reports):
            print(f"({x},{y}) frequency: {x*4_000_000+y}")

0/23: searching at dist=1412862 from sensor [325337, 2568863]
1/23: searching at dist=1606122 from sensor [3988825, 837820]
2/23: searching at dist=1689580 from sensor [1611311, 2053174]
3/23: searching at dist=1336118 from sensor [101890, 3940049]
4/23: searching at dist=310581 from sensor [3962702, 2558425]
5/23: searching at dist=711968 from sensor [2957890, 2160813]
(3257428,2573243) frequency: 13029714573243
6/23: searching at dist=310473 from sensor [3907456, 3325610]
7/23: searching at dist=551591 from sensor [3354177, 3435919]
8/23: searching at dist=455664 from sensor [3997379, 3071868]
9/23: searching at dist=948843 from sensor [145143, 1714962]
10/23: searching at dist=652560 from sensor [611563, 3148864]
11/23: searching at dist=1294221 from sensor [3080405, 3904777]
12/23: searching at dist=585058 from sensor [644383, 10732]
13/23: searching at dist=516998 from sensor [3229566, 1694167]
14/23: searching at dist=1172536 from sensor [1600637, 3984884]
15/23: searching at dis